In [7]:
import os.path
import re
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the scope for accessing Gmail
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail():
    """
    Authenticates the user with Gmail API and returns the credentials.
    """
    creds = None
    # Load credentials from token.json if it exists
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no valid credentials, prompt the user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES, redirect_uri='http://localhost:8080/')
            creds = flow.run_local_server(port=8080)
        # Save the credentials for future use
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds

def search_emails(service, query):
    """
    Searches emails in the Gmail account based on the query and extracts email addresses from the 'To' field.
    """
    results = service.users().messages().list(userId='me', q=query).execute()
    messages = results.get('messages', [])

    email_addresses = set()

    if not messages:
        print('No messages found.')
    else:
        for msg in messages:
            msg_id = msg['id']
            message = service.users().messages().get(userId='me', id=msg_id).execute()
            payload = message['payload']
            headers = payload.get('headers', [])
            for header in headers:
                if header['name'] == 'To':
                    emails = header['value']
                    email_list = re.findall(r'\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b', emails, re.I)
                    email_addresses.update(email_list)
    
    return email_addresses

def save_to_csv(email_addresses, filename='email_addresses.csv'):
    """
    Saves the set of email addresses to a CSV file.
    """
    df = pd.DataFrame(list(email_addresses), columns=['Email Address'])
    df.to_csv(filename, index=False)
    print(f"Email addresses saved to {filename}")

def main():
    """
    Main function to authenticate and search for emails containing a specific word, then save the results to a CSV file.
    """
    creds = authenticate_gmail()
    service = build('gmail', 'v1', credentials=creds)

    # Modify 'specific_word' to the word you want to search for in the subject field of the sent emails.
    specific_word = 'server'
    query = f'subject:{specific_word} in:sent'

    email_addresses = search_emails(service, query)
    
    print("Email Addresses found:")
    for email in email_addresses:
        print(email)
    
    # Save the email addresses to a CSV file
    save_to_csv(email_addresses)

if __name__ == '__main__':
    main()


Email Addresses found:
wanglangtoronto@gmail.com
info@landwer.ca
reservations@modusristorante.com
info@alobaryorkville.com
1to-reservations@1hotels.com
info@elreybar.com
socialmedia@firkinpubs.com
info@pizzarustica.ca
info@vitbeo.com
info@thebeastrestaurant.com
hello@praisebottleshop.com
piebar@fabrestaurants.ca
Crystal@foxcityplace.com
inquiry@daldongnaebbq.com
goodman@fabrestaurants.ca
sisterscotoronto@gmail.com
hello@beavertails.com
thepourboymanning@gmail.com
contactus@pizzerialibretto.com
pigeons@pigeonespressobar.com
hello@cockadoodledoo.ca
rosesnewyork@gmail.com
comment@gabbys.ca
reservation@cktl.ca
info@overpressureclub.com
john@johnandsonsoysterhouse.com
info@thecaledonian.ca
cm@chinchin.com
customerservice@kernelspopcorn.com
careers@alofoodgroup.com
wc5301@wingcitybystlouis.ca
Toronto@princestreetpizza.com
254bsmt@gmail.com
gigionbloor@gmail.com
pearlharbourfront@gmail.com
info@bridgettebartoronto.com
hello@tapigotapioca.com
feedback@mrgreek.com
staff@santamarta.ca
Joseph@ckt